# Migrando do SQL para o Pandas

Eu acho a documentação do Pandas sensacional e tem um item que ajudou muito em converter meu raciocínio forjado no SQL para os objetos do Pandas.

![](http://fourb.com.br/blog/wp-content/uploads/2020/03/sql-pandas-768x412.png)

Estou falando da página “Comparison with other tools”, que além do SQL mostra exemplos de R, SAS e Stata de forma simples e direta.

Vou mostrar alguns exemplos, mas recomendo a leitura diretamente na documentação para uma visão mais abrangente.

##Dataframe

Antes de começar, vamos considerar o seguinte dataframe para estudo:





In [0]:
# importando as dependencias e criando o dataframe de exemplo
import pandas as pd
import numpy as np


In [28]:

data = {
          'total_bill': [16.99, 10.34, 21.01, 23.68, 24.59, 12.56, 16.99, 10.34, 21.01, 23.68, 24.59, 12.56],
          'tip': [1.01, 1.66, 3.5, 3.31, 3.61, 2, 1.01, 1.66, 3.5, 3.31, 3.61, 2],
          'smoker': ['no', 'no','no','no','no','no', 'no', 'no','no','no','no','no'],
          'time': ['Dinner', 'Lunch', 'Lunch', 'Dinner', 'Dinner', 'Lunch', 'Dinner', 'Dinner', 'Dinner', 'Lunch', 'Dinner', 'Dinner'],
          'sex': ['Male', 'Male', 'Female', 'Male', 'Male', 'Male', 'Male', 'Male', 'Female', 'Male', 'Female', 'Female'],
          'day': ['Fri', 'Sat', 'Sun', 'Thur', 'Mon', 'Sun', 'Thur', 'Mon', 'Fri', 'Sat', 'Sun', 'Thur']
        }

tips = pd.DataFrame(data,columns=['total_bill',  'tip',  'smoker', 'time', 'sex', 'day'])
tips.head()

,total_bill,tip,smoker,time,sex,day
0,16.99,1.01,no,Dinner,Male,Fri
1,10.34,1.66,no,Lunch,Male,Sat
2,21.01,3.50,no,Lunch,Female,Sun
3,23.68,3.31,no,Dinner,Male,Thur
4,24.59,3.61,no,Dinner,Male,Mon


## Select
Com certeza, a primeira palavra que digitamos quando começamos no SQL. Com este comando de DML, buscamos selecionar informações da base de dados. Podemos informar do SGDB quais as colunas a serem retornadas ou trazer todas usando o popular select * from table. No exemplo abaixo, buscamos algumas colunas da tabela tips, limitando a 5 linhas de retorno:


```
SELECT total_bill, tip, smoker, time
FROM tips
LIMIT 5;
```

No Pandas o mesmo resultado é obtido passa uma lista de nomes de colunas para nosso DataFrame:

In [29]:
tips[['total_bill', 'tip', 'smoker', 'time']].head(5)

,total_bill,tip,smoker,time
0,16.99,1.01,no,Dinner
1,10.34,1.66,no,Lunch
2,21.01,3.50,no,Lunch
3,23.68,3.31,no,Dinner
4,24.59,3.61,no,Dinner


##Where
As vezes esquecido nos updates e deletes (brincadeira!), a cláusula WHERE no SQL especifica que uma instrução deve afetar apenas as linhas que atendem aos critérios especificados. No exemplo abaixo, queremos ver todas as linhas da tabela tips onde o time for igual a dinner



```
SELECT *
FROM tips
WHERE time = 'Dinner'
LIMIT 5;
```
No Pandas, o DataFrame pode ser filtrado de várias maneiras. A mais intuitiva é usando a indexação booleana, que não vem ao caso agora. Vamos ao código:


In [30]:
tips[tips['time'] == 'Dinner'].head(5)

,total_bill,tip,smoker,time,sex,day
0,16.99,1.01,no,Dinner,Male,Fri
3,23.68,3.31,no,Dinner,Male,Thur
4,24.59,3.61,no,Dinner,Male,Mon
6,16.99,1.01,no,Dinner,Male,Thur
7,10.34,1.66,no,Dinner,Male,Mon


## Group By
O GROUP BY é utilizado para separar os registros selecionados em grupos específicos. Com os agrupamentos criados, podemos aplicar as funções de agregação, que podem ser contar os registros do grupo, somar valores, valores máximos e mínimos e etc.

**Vamos a um exemplo**

No código abaixo, **contamos** os registros agrupados pela coluna sex.


```
SELECT sex, count(*)
FROM tips
GROUP BY sex;
/*
Female     4
Male       8
*/ 
```
O equivalente no Pandas pode ser:


In [31]:
tips.groupby('sex').size()

sex
Female    4
Male      8
dtype: int64

Perceba que no pandas usamos **size()** e não count(). Isso acontece porque count() é aplicado a cada coluna, retornando o número de registros não nulos de cada uma.

Também é possível a busca de várias agregações em uma única instrução. Nesse caso, devemos utilizar a função **agg()** e passar um dicionário com as colunas e a função desejada. Bem de boa:


```
SELECT day, AVG(tip), COUNT(*)
FROM tips
GROUP BY day;
/*
Fri   2.734737   19
Sat   2.993103   87
Sun   3.255132   76
Thur  2.771452   62
*/
```



In [32]:
tips.groupby('day').agg({'tip': np.mean, 'day': np.size})

,tip,day
day,,
Fri,2.255000,2
Mon,2.635000,2
Sat,2.485000,2
Sun,3.036667,3
Thur,2.106667,3


## Inner Join
Utilizadíssimo nos selects, o inner join é um tipo de junção que busca apenas os dados que existem nas duas tabelas juntadas. Vamos considerar os seguintes DataFrames como exemplo:

In [34]:
df1 = pd.DataFrame({
                    'key': ['A', 'B', 'C', 'D'],
                    'value': np.random.randn(4)
                    })
df1.head()

,key,value
0,A,-0.294990
1,B,0.440535
2,C,0.617567
3,D,-0.114170


In [38]:
df2 = pd.DataFrame({
                    'key': ['B', 'D', 'D', 'E'],
                    'value': np.random.randn(4)
                    })
df2.head()

,key,value
0,B,-0.035947
1,D,0.467819
2,D,0.554744
3,E,-0.863968


Assumimos que temos as mesmas tabelas na base com a mesma estrutura


```
SELECT *
FROM df1
INNER JOIN df2
  ON df1.key = df2.key;
```



In [39]:
# merge() realiza um INNER JOIN por default
pd.merge(df1, df2, on='key')

,key,value_x,value_y
0,B,0.440535,-0.035947
1,D,-0.114170,0.467819
2,D,-0.114170,0.554744


In [0]:
O método merge()permite também que o join seja feito por colunas com nomes diferentes:

In [41]:
 df1 = pd.DataFrame({'fruta': ['maçã', 'laranja', 'pera', 'pêssego'],
                    'value': [1, 2, 3, 5]})
 
 df2 = pd.DataFrame({'alimento': ['maçã', 'pão', 'pera', 'pêssego'],
                    'value': [5, 6, 7, 8]})
 
 df1.merge(df2, left_on='fruta', right_on='alimento', 
                        suffixes=('_fruta', '_alimento'))

,fruta,value_fruta,alimento,value_alimento
0,maçã,1,maçã,5
1,pera,3,pera,7
2,pêssego,5,pêssego,8


## Left (Outer) Join
O left join traz todos os registros da primeira tabela declarada (da esquerda) e se houverem dados tabela relacionada também serão projetados. No exemplo, utilizo o mesmo data frame do inner join. No pandas, o tipo de junção é passada no parâmetro how do merge().



```
-- mostra todos os registros de df1
SELECT *
FROM df1
LEFT OUTER JOIN df2
  ON df1.key = df2.key;
```



In [48]:
df1 = pd.DataFrame({
                    'key': ['A', 'B', 'C', 'D'],
                    'value': np.random.randn(4)
                    }) 

df2 = pd.DataFrame({
                    'key': ['B', 'D', 'D', 'E'],
                    'value': np.random.randn(4)
                    })


# mostra todos os registros de df1
pd.merge(df1, df2, on='key', how='left')

,key,value_x,value_y
0,A,-0.671132,NaN
1,B,-0.963374,-0.442932
2,C,-1.044864,NaN
3,D,-0.794477,-0.618664
4,D,-0.794477,1.051263


## Union
No pandas temos o mesmo resultado usando a função concat().

In [0]:
df1 = pd.DataFrame({
                    'city': ['Chicago', 'San Francisco', 'New York City'],
                     'rank': range(1, 4)
                   })

df2 = pd.DataFrame({
                    'city': ['Chicago', 'Boston', 'Los Angeles'],
                    'rank': [1, 4, 5]
                  })



```
SELECT city, rank
FROM df1
UNION ALL
SELECT city, rank
FROM df2;
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
      Chicago     1
       Boston     4
  Los Angeles     5
*/
```



In [51]:
pd.concat([df1, df2])

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


Em SQL, UNION é similar do UNION ALL, porém o UNION remove os registros duplicados.


```
SELECT city, rank
FROM df1
UNION
SELECT city, rank
FROM df2;
-- Perceba que agora aparece uma única vez a cidade de Chicago
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
       Boston     4
  Los Angeles     5
*/
```

No Pandas, o método drop_duplicates() pode ser usado em conjunto com o concat() para repetir esse comportado.

In [52]:
pd.concat([df1, df2]).drop_duplicates()

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5
